# <span style='color:Blue'>Question 3</span>
### <span style='color:Blue'>1. Decision Tree Implementation (impurity measure: entropy)</span>
### <span style='color:Blue'>2. K-Fold Cross Validation (K=10)</span>
### <span style='color:Blue'>3. Improvement Strategies (Gini-Index and Post-Pruning)</span>
***
**NOTE:**
<p style='text-align: justify;'>
1. Below mentioned code is the combined implementation of Decision Tree for a classification problem using entropy and gini as choices for the impurity measure in the tree. You just need to pass appropriate name of the impurity_measure to be used in the tree.</p>

<p style='text-align: justify;'>
2. Comments have been added in every module to explain the concept used in the implementation for the very module.</p>

<p style='text-align: justify;'>
3. Decision Tree Representation is done using a nested dictionary where each key value is a node and values are the left (0th index) and right child (1st index) which can be further nested in case of internal nodes/ decision nodes.</p>

<p style='text-align: justify;'>
4. To keep the implementations combined, rather than finding the min-gini-index, max is found out after negating the weighted gini-index so that same logic for both entropy and gini-index can be used in the splitting of the node.</p>

<p style='text-align: justify;'>
5. Random shuffling on the data is done before creating chunks. The whole data is used to create 10 chunks from which one is used for validation and remanining for training. The reports at the bottom contains validation set accuracy (averaged over K runs) for initial implementation (using entropy) and improved implementation (using gini-index).</p>

<p style='text-align: justify;'>
6. Two result files are generated containing averaged validation accuracies for both implementations.</p>

<p style='text-align: justify;'>
7. Filenames for the result files have been changed to distinguish between the files. </p>

## _Implementation using Python_

In [1]:
# FoML Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.
import csv
import random
import numpy as np
import time

random.seed(4)
np.random.seed(4)

# Enter You Name Here
myname = "Kuldeep Gautam"

In [2]:
# Implement your decision tree below
class DecisionTree():

    def __init__(self, attr_names, impurity_measure):
        super(DecisionTree, self).__init__()
        self.tree = {}
        # List of attribute names, used for creating nodes in the decision tree as well as in prediction.
        self.attr_names = attr_names
        # Type of impurity measure to be used in building the decision tree.
        self.impurity_measure = impurity_measure
    
    
    # Computing the potential threshold values for every column for creating best splits.       
    def compute_threshold_values(self, data, threshold_values):
        target = data[:, -1]
        for column_idx in range(data.shape[1]-1):
            if not threshold_values[column_idx]==False:
                temp_thr_list = []
                for idx in range(len(target)-1):
                    # Only considering those indexes whereever there is a class change in the target column.
                    if target[idx] != target[idx+1]:
                        thr = (data[idx, column_idx] + data[idx, column_idx])/2
                        temp_thr_list.append(thr)

                threshold_values[column_idx] = list(set(temp_thr_list))
        return threshold_values

    
    # Computing the entropy measure for a set of data points.
    def get_entropy(self, data):
        targets = data[:,-1]
        classes, classes_count = np.unique(targets, return_counts=True)

        # Case of min entropy
        if len(classes) == 1:
            entropy = float(0)

        # Case of max entropy
        elif len(classes) == 2 and len(set(classes_count)) == 1:
            entropy = float(1)

        # Other cases
        else:
            class_probs = classes_count/sum(classes_count)
            entropy = sum(-1 * class_probs * np.log2(class_probs))

        return entropy

    
    # Computing Gini-Indedx measure for a set of data points.
    def get_gini(self, data):
        targets = data[:,-1]
        classes, classes_count = np.unique(targets, return_counts=True)

        if len(classes) == 1:
            gini = float(0)

        elif len(classes) == 2 and len(set(classes_count)) == 1:
            gini = 0.5

        else:
            class_probs = classes_count/sum(classes_count)
            gini = 1 - sum(np.square(class_probs))

        return gini

    
    # Computing the weighted child entropy or weighted child gini-index values using the probabilities.
    def get_weighted_impurity(self, left_dtree, right_dtree):
        possible_outcomes = left_dtree.shape[0] + right_dtree.shape[0]
        prob_left = left_dtree.shape[0]/possible_outcomes
        prob_right = right_dtree.shape[0]/possible_outcomes

        if self.impurity_measure == 'entropy':
            weighted_impurity = prob_left*self.get_entropy(left_dtree) + prob_right*self.get_entropy(right_dtree)

        else:
            weighted_impurity = prob_left*self.get_gini(left_dtree) + prob_right*self.get_gini(right_dtree)
        
        return weighted_impurity

    
    # Finding the best attribute to create a split. Note that to keep the implementations combined, rather than
    # finding the min-weighted gini-index for split max of -(weighted gini-index) is found.
    def get_best_attr(self, data, threshold_values):
        best_attr_data = {}
        best_gain = -1*np.inf
        
        if self.impurity_measure == 'entropy':
            parent_impurity = self.get_entropy(data)
            
        else:
            parent_impurity = float(0)

        # Calculation of entropy remains same even if parent column exists in the data. 
        # Just the number of rows should be filtered after selecting best attribute.
        for col_idx, thr_values in threshold_values.items():
            if not thr_values==False:
                for thr in thr_values:
                    # Split data on thr
                    left_dtree, right_dtree = data[data[:, col_idx] <= thr], data[data[:, col_idx] > thr]

                    # Compute best info gain/gini-index
                    if left_dtree.shape[0] > 0 and right_dtree.shape[0] > 0:
                        weighted_child_impurity = self.get_weighted_impurity(left_dtree, right_dtree)
                        info_gain  =  parent_impurity - weighted_child_impurity

                        if info_gain > best_gain:
                            best_attr_data['best_attr_idx'] = col_idx
                            best_attr_data['best_attr_thr'] = thr
                            best_gain = info_gain
        
        return best_attr_data, best_gain

    
    # Heart of the algorithm which builds the decision tree based on based splits and choosed impurity measure.
    def learn(self, data, threshold_values):
        uniq_targets, count_targets = np.unique(data[:, -1], return_counts=True)

        # If it is pure node or no more attributes left, return the max class label
        if self.impurity_measure=='entropy' and (self.get_entropy(data) == float(0) or not any(threshold_values.values())==True):
            return uniq_targets[np.argmax(count_targets)]

        # If not a pure node but contains equal number of classes, return 0 as class label (default choice)
        elif self.impurity_measure=='entropy' and (self.get_entropy(data) == float(1) or not any(threshold_values.values())==True): 
            return float(0)

        elif self.impurity_measure=='gini' and (self.get_gini(data) == float(0) or not any(threshold_values.values())==True):
            return uniq_targets[np.argmax(count_targets)]

        # If not a pure node but contains equal number of classes, return 0 as class label (default choice)
        elif self.impurity_measure=='gini' and (self.get_gini(data) == 0.5 or not any(threshold_values.values())==True):       
            return float(0)

        else:
            threshold_values = self.compute_threshold_values(data, threshold_values)
            best_attr_data, best_gain = self.get_best_attr(data, threshold_values)

            # In case of leaf node, best_gain var is not updated and hence to avoid that, this condition is put.
            # Taking condition >-1 works for combined implementation of entropy and gini.
            # Min value for gini can be -1 (for this implementation) and entropy will be > 0 if attributes are still left)
            if best_gain > -1:
                best_attr_idx = best_attr_data['best_attr_idx']
                best_attr_thr = best_attr_data['best_attr_thr']
                node_condition = '{} <= {}'.format(self.attr_names[best_attr_idx], best_attr_thr)
                dtree = {node_condition: []}
                child_threshold_values = threshold_values.copy()
                child_threshold_values[best_attr_idx] = False

                left_dtree  = data[data[:, best_attr_idx] <= best_attr_thr]
                right_dtree  = data[data[:, best_attr_idx] > best_attr_thr]
                dtree[node_condition].append(self.learn(left_dtree, threshold_values=child_threshold_values))
                dtree[node_condition].append(self.learn(right_dtree, threshold_values=child_threshold_values))
                return dtree
        

    # Classify/Predict function to make predictions using the learnt decision tree.
    def classify(self, test_instance, tree):
        node_condition = next(iter(tree.keys()))
        attr_name, thr = node_condition.split(' <= ')

        if test_instance[self.attr_names.index(attr_name)] <= float(thr):
            result = tree[node_condition][0] # left child

        else:
            result = tree[node_condition][1] # right child

        # Checking if the result is itself a dictionary i.e, a decision node. If yes, keep recusring.
        if isinstance(result, dict):
            return self.classify(test_instance, result)

        else:
            return result

        
# Main function building decision tree and performing classification
def run_decision_tree(impurity_measure):
    # Load data set
    with open("wine-dataset.csv") as f:
        # next(f, None)
        attr_names = f.readline().split(',')[:-1]
        data = [tuple(line) for line in csv.reader(f, delimiter=",", quoting=csv.QUOTE_NONNUMERIC)]

    print("Number of records: %d" % len(data))

#     Split into training/test sets
#     training_set = [x for i, x in enumerate(data) if i % K != 9]
#     test_set = [x for i, x in enumerate(data) if i % K == 9]
    
    # K-Fold Cross Validation. (K=10)
    K = 10
    fold_size = len(data)//K
    kfold_cv_acc = []
    t = []
    tree = DecisionTree( attr_names=attr_names, impurity_measure=impurity_measure )

    random.shuffle(data)

    for i in range(K):
        print('\nFOLD-{}'.format(i+1) + '='*10)
        start = i*fold_size
        end = start + fold_size
        # valid_set = training_set[start: end]
        # train_set = training_set[:start] + training_set[end:]
        valid_set = data[start: end]
        train_set = data[:start] + data[end:]

        threshold_values = {}
        for idx in range(len(attr_names)):
            threshold_values[idx] = True

        # Construct a tree using training set
        start = time.time()
        tree.tree = tree.learn( np.array(train_set), threshold_values )
        end = time.time()-start
        t.append(end)

        # Validate results on validation set
        val_results = []
        for instance in valid_set:
            result = tree.classify( np.array(instance[:-1]), tree.tree )
            val_results.append( result == instance[-1] )

        val_accuracy = val_results.count(True)/len(val_results)
        print("val accuracy: %.4f " % val_accuracy)
        kfold_cv_acc.append(val_accuracy)

    accuracy = sum(kfold_cv_acc)/K
    print('\nK-Fold CV Accuracy: {:.4f}'.format(accuracy))
    print('Time taken in learning Decision Tree using {}: {:.3f} seconds'.format(impurity_measure, sum(t)/K))
    
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"-{}-result.txt".format(impurity_measure), "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()

In [3]:
if __name__ == "__main__":
    run_decision_tree('entropy')

Number of records: 4898

FOLD-1==========
val accuracy: 0.8016 

FOLD-2==========
val accuracy: 0.8098 

FOLD-3==========
val accuracy: 0.7975 

FOLD-4==========
val accuracy: 0.7791 

FOLD-5==========
val accuracy: 0.7853 

FOLD-6==========
val accuracy: 0.8200 

FOLD-7==========
val accuracy: 0.7832 

FOLD-8==========
val accuracy: 0.8384 

FOLD-9==========
val accuracy: 0.8221 

FOLD-10==========
val accuracy: 0.7812 

K-Fold CV Accuracy: 0.8018
Time taken in learning Decision Tree using entropy: 2.131 seconds


In [4]:
if __name__ == "__main__":
    run_decision_tree('gini')

Number of records: 4898

FOLD-1==========
val accuracy: 0.8241 

FOLD-2==========
val accuracy: 0.7710 

FOLD-3==========
val accuracy: 0.8528 

FOLD-4==========
val accuracy: 0.7955 

FOLD-5==========
val accuracy: 0.8037 

FOLD-6==========
val accuracy: 0.8098 

FOLD-7==========
val accuracy: 0.7935 

FOLD-8==========
val accuracy: 0.7996 

FOLD-9==========
val accuracy: 0.7996 

FOLD-10==========
val accuracy: 0.8037 

K-Fold CV Accuracy: 0.8053
Time taken in learning Decision Tree using gini: 2.143 seconds


# Accuracy Report
### 1. Accuracy for initial implementation (using entropy as impurity)
The averaged accuracy reports on validation set for the initial implementation using entropy as impurity as an impurity measure came out to be **80.18%**. This is a decennt accuracy for an initial implementation just taking into consideration random shuffling of data, binary splits, entropy and information gain.
### 2. Accuracy of improved implementation
#### 2a. Gini-Index
There is not much improvement in the averaged accuracy by using gini-index as impurity measure. The reported accuracy on validation set is **80.53** Minute difference in the accuracy can be seen. It may be possible that significant difference starts coming into the picute once the data increases in size and then time would also show certain difference. The way gain is computed, it takes less time in computig gini index as it does not take into consideration the impurity of parent node.
#### 2b. Post Pruning
The implementation for this was in between but could not be done on time. Here are some general thoughts on how this could improve the performance.
1. Post pruning is a famous technique for avoiding overfitting in decision trees and hence the model will be able to perform better in terms of generalization error.
2. The process goes like: it starts with finding out what are leaf nodes and decision nodes that has been misclassified by the algorithm. The idea is to prune the tree by removing such misclassifications from the tree and replacing them with the mostly occuring lead node value in the dataset.